# 🔄 Spotify Library Sync

This notebook downloads your Spotify library and saves it locally for offline analysis.

**What it does:**
- ✅ Fetches all your playlists (owned only)
- ✅ Fetches your Liked Songs (❤️ master playlist)
- ✅ Downloads track and artist metadata
- ✅ Saves everything to `../data/` as parquet files
- ✅ Incremental updates (only fetches changes)

**Run this first!** Then use `02_analyze_library.ipynb` for analysis.

**💡 Tip:** For automated daily syncs, use `scripts/spotify_sync.py` instead (configured via cron job). See `README.md` for details.

## 1️⃣ Setup

Install dependencies and configure credentials.

In [12]:
# Install dependencies (run once)
%pip install -q pandas spotipy pyarrow tqdm python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [13]:
# Add project to path
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"✅ Project root: {PROJECT_ROOT}")

✅ Project root: /Users/aryamaan/Desktop/Projects/spotim8


In [14]:
import os
from dotenv import load_dotenv

# Load credentials from ../.env file
env_path = PROJECT_ROOT / ".env"
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Loaded credentials from {env_path}")
else:
    print(f"⚠️  No .env file found at {env_path}")
    print("   Create one with SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, SPOTIPY_REDIRECT_URI")

# Verify credentials are set
client_id = os.environ.get("SPOTIPY_CLIENT_ID", "")
if client_id and client_id != "YOUR_CLIENT_ID":
    print(f"   Client ID: {client_id[:8]}...")
else:
    print("   ❌ SPOTIPY_CLIENT_ID not set!")

✅ Loaded credentials from /Users/aryamaan/Desktop/Projects/spotim8/.env
   Client ID: 8263fcc5...


## 2️⃣ Connect to Spotify

This will open a browser window for authentication on first run.

In [15]:
from spotim8 import Spotim8, set_response_cache
from spotim8.catalog import CacheConfig

# Data directory (stores downloaded data)
DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

# Enable API response caching to avoid rate limits
# Cached responses are reused for 1 hour
API_CACHE_DIR = DATA_DIR / ".api_cache"
set_response_cache(API_CACHE_DIR, ttl=3600)

# Initialize client with caching
sf = Spotim8.from_env(
    progress=True,
    cache=CacheConfig(dir=DATA_DIR)
)

print(f"✅ Connected to Spotify!")
print(f"📁 Data will be saved to: {DATA_DIR}")

📦 API response cache enabled: /Users/aryamaan/Desktop/Projects/spotim8/data/.api_cache (TTL: 3600s)
✅ Connected to Spotify!
📁 Data will be saved to: /Users/aryamaan/Desktop/Projects/spotim8/data


## 3️⃣ Sync Your Library

This fetches your playlists and tracks. First run may take a few minutes.

In [16]:
# Sync library (incremental - only fetches changes)
stats = sf.sync(
    owned_only=True,           # Only your playlists, not followed ones
    include_liked_songs=True   # Include Liked Songs as master playlist
)

print(f"\n📊 Sync complete!")

🔄 Starting library sync...
✅ All playlists up to date!
✅ Sync complete! Checked 327 playlists, updated 0, added 0 track entries

📊 Sync complete!


## 4️⃣ Build Full Data Tables

Now let's build all the detailed tables (tracks, artists, etc.)

In [17]:
# Fetch all data tables (uses cache if available)
print("📥 Building data tables...\n")

playlists = sf.playlists()
print(f"✅ Playlists: {len(playlists):,}")

playlist_tracks = sf.playlist_tracks()
print(f"✅ Playlist-track links: {len(playlist_tracks):,}")

tracks = sf.tracks()
print(f"✅ Unique tracks: {len(tracks):,}")

track_artists = sf.track_artists()
print(f"✅ Track-artist links: {len(track_artists):,}")

artists = sf.artists()
print(f"✅ Artists: {len(artists):,}")

# Build the wide table (everything joined)
library = sf.library_wide()
print(f"✅ Library wide table: {len(library):,} rows")

📥 Building data tables...

✅ Playlists: 757
✅ Playlist-track links: 44,102
✅ Unique tracks: 5,270
✅ Track-artist links: 8,504
✅ Artists: 2,610
✅ Library wide table: 44,347 rows


## 5️⃣ View Your Data

In [18]:
# Show status summary
sf.print_status()


        SPOTIM8 DATA STATUS
📁 Cache directory: /Users/aryamaan/Desktop/Projects/spotim8/data
👤 User: 31iol2qamank24owygxo7kpq533y
🕐 Last sync: 2025-12-24T21:58:44.934198+00:00

📊 Cached data:
   • Playlists: 757
   • Playlist tracks: 44,102
   • Unique tracks: 5,270
   • Track-artist links: 8,504
   • Artists: 2,610



In [19]:
# Preview playlists
print("📂 Your Playlists:")
playlists[["name", "track_count", "is_liked_songs", "is_owned"]].head(15)

📂 Your Playlists:


,name,track_count,is_liked_songs,is_owned
0,❤️ Liked Songs,5115,True,True
1,OtherDec25,27,False,True
2,DanceDec25,6,False,True
3,HipHopDec25,15,False,True
4,Dec25,47,False,True
5,AJamLatin,26,False,True
6,AJamIndie,121,False,True
7,AJamRock,125,False,True
8,AJamPop,348,False,True
9,AJamR&B/Soul,357,False,True


In [20]:
# Preview tracks
print("🎵 Sample Tracks:")
tracks[["name", "album_name", "popularity", "duration_ms"]].head(10)

🎵 Sample Tracks:


,name,album_name,popularity,duration_ms
0,Figaro,Madvillainy,59,145706
1,Meat Grinder,Madvillainy,64,131866
2,Rhymes Like Dimes,Operation: Doomsday (Complete),66,258613
3,Rapp Snitch Knishes,MM..FOOD,74,172893
4,All Caps,Madvillainy,70,130479
5,Jeep (feat. Terror Reid),I’m Not Supposed To Be Here,58,120896
6,Cannonball (feat. Don Toliver),Euphoria,72,122568
7,Weak,Promised Land,67,202560
8,Jungle,USB,62,198805
9,Beto’s Horns - fred remix,USB,64,226222


In [21]:
# Preview artists
print("🎤 Top Artists (by followers):")
artists.nlargest(10, "followers")[["name", "genres", "popularity", "followers"]]

🎤 Top Artists (by followers):


,name,genres,popularity,followers
1160,Arijit Singh,"[hindi pop, bollywood, desi, bangla pop]",92,168800957
304,Taylor Swift,[],100,148493497
786,Ed Sheeran,[soft pop],90,123881028
161,Billie Eilish,[],93,121012260
575,The Weeknd,[],96,115913722
254,Ariana Grande,[pop],95,108576516
636,Eminem,"[rap, hip hop]",91,106037116
1097,Bad Bunny,"[reggaeton, trap latino, urbano latino, latin]",98,105279863
77,Drake,[rap],98,105142837
783,Justin Bieber,[],94,85978354


## 6️⃣ Check Saved Files

In [22]:
# List saved files
print(f"📁 Files in {DATA_DIR}:\n")
for f in sorted(DATA_DIR.glob("*.parquet")):
    size_kb = f.stat().st_size / 1024
    print(f"   {f.name:30} {size_kb:>8.1f} KB")

📁 Files in /Users/aryamaan/Desktop/Projects/spotim8/data:

   artists.parquet                   198.0 KB
   library_wide.parquet             1969.4 KB
   playlist_tracks.parquet           624.1 KB
   playlists.parquet                 124.3 KB
   track_artists.parquet             217.0 KB
   tracks.parquet                    610.8 KB


---

## ✅ Done!

Your library is now saved locally. Next steps:

1. **Analyze**: Open `02_analyze_library.ipynb` for visualizations
2. **Playlist Analysis**: Open `03_playlist_analysis.ipynb` for genre clustering
3. **Re-sync**: Run this notebook again anytime to fetch new changes

The data is cached, so future runs are fast!